In [1]:
import pickle
import sys
import mdtraj as md
import os
import numpy as np
from simtk import unit
from time import time

In [2]:
from coddiwomple.openmm.states import OpenMMParticleState, OpenMMPDFState
from coddiwomple.openmm.propagators import *

from perses.annihilation.lambda_protocol import RelativeAlchemicalState

from coddiwomple.openmm.reporters import OpenMMReporter
import os
from coddiwomple.particles import Particle
from perses.dispersed.feptasks import minimize
import tqdm
_OPENMM_ENERGY_UNIT = unit.kilojoules_per_mole

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
RDKit WARNING: [15:47:13] Enabling RDKit 2019.09.3 jupyter extensions


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU


In [3]:
pickl = f"benzene_fluorobenzene.vacuum.factory.pkl"

with open(pickl, 'rb') as f:
    factory = pickle.load(f)

In [ ]:


pickl = f"benzene_fluorobenzene.vacuum.factory.pkl"

with open(pickl, 'rb') as f:
    factory = pickle.load(f)


# In[ ]:


hybrid_system = factory.hybrid_system
hybrid_positions = factory.hybrid_positions


# Now we can load this system into an `OpenMMPDFState` and load the positions into an `OpenMMParticleState`

# In[ ]:


from coddiwomple.openmm.states import OpenMMParticleState, OpenMMPDFState
from coddiwomple.openmm.propagators import *


# In[ ]:


from perses.annihilation.lambda_protocol import RelativeAlchemicalState
particle_state = OpenMMParticleState(positions = hybrid_positions, box_vectors = np.array(hybrid_system.getDefaultPeriodicBoxVectors()))
pdf_state = OpenMMPDFState(system = hybrid_system, alchemical_composability = RelativeAlchemicalState, pressure=None)


langevin_integrator = OpenMMLangevinIntegrator(temperature = pdf_state.temperature, timestep = 2.0 * unit.femtoseconds)


# In[ ]:


endstate_propagator = OpenMMBaseIntegrationPropagator(openmm_pdf_state = pdf_state, integrator = langevin_integrator)


# In[ ]:


from coddiwomple.openmm.reporters import OpenMMReporter
import os
reporter = OpenMMReporter('test_fluoro', 'solvent', md_topology = factory.hybrid_topology)


# In[ ]:


from coddiwomple.particles import Particle
particle = Particle(0)
particle.update_state(particle_state)
reporter.record([particle])


# In[ ]:


from perses.dispersed.feptasks import minimize


# In[ ]:


minimize(endstate_propagator.pdf_state, particle_state)


# In[ ]:

import tqdm
for i in tqdm.trange(150):
    particle_state, _return_dict = endstate_propagator.apply(particle_state, n_steps = 500)
    reporter.record([particle], save_to_disk=False)


# In[ ]:


reporter.record([particle], save_to_disk=True)



In [4]:
traj = md.Trajectory.load(os.path.join(os.getcwd(), 'test_fluoro', 'solvent.neq.pdb'))

    
hybrid_system = factory.hybrid_system
num_frames = traj.n_frames
pdf_state = OpenMMPDFState(system = factory.hybrid_system, alchemical_composability = RelativeAlchemicalState, pressure=None)

DEBUG:openmm_states:successfully instantiated OpenMMPDFState equipped with the following parameters: {'lambda_electrostatics': None, 'lambda_electrostatics_insert': 0.0, 'lambda_sterics_insert': 0.0, 'lambda_electrostatics_delete': 0.0, 'lambda_bonds': 0.0, 'lambda_sterics_delete': 0.0, 'lambda_sterics_core': 0.0, 'lambda_electrostatics_core': 0.0, 'lambda_torsions': 0.0, 'lambda_angles': 0.0, 'lambda_sterics': None}


In [5]:
import copy
from coddiwomple.openmm.integrators import OMMLIAIS
from coddiwomple.openmm.propagators import OMMAISP

In [6]:
n_steps = 100
x = 'fractional_iteration'
alchemical_functions = {
                         'lambda_sterics_core': x,
                         'lambda_electrostatics_core': x,
                         'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                         'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                         'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                         'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                         'lambda_bonds': x,
                         'lambda_angles': x,
                         'lambda_torsions': x}

ais_integrator = OMMLIAIS(
                         alchemical_functions,
                         n_steps,
                         temperature=300.0 * unit.kelvin,
                         collision_rate=1.0 / unit.picoseconds,
                         timestep=1.0 * unit.femtoseconds,
                         splitting="P I H N S V R O R V",
                         constraint_tolerance=1e-6)



try:
    os.system(f"rm -r fluoro_bounce")
except:
    pass

reporter = OpenMMReporter('fluoro_bounce', 'runner', factory.hybrid_topology)
ais_propagator = OMMAISPR(openmm_pdf_state = pdf_state, integrator = ais_integrator, record_state_work_interval = 1, reassign_velocities=True, reporter=reporter)

master_works = []
frames = [33, 45, 10, 34, 100, 101, 54, 76]
import tqdm
for num_trajectories in tqdm.trange(7):
    positions = traj.xyz[frames[num_trajectories]] * unit.nanometers
    particle_state = OpenMMParticleState(positions=positions, box_vectors=traj.unitcell_vectors[frames[num_trajectories]]*unit.nanometers)

    _, return_dict = ais_propagator.apply(particle_state, n_steps=n_steps, apply_pdf_to_context = True, reset_integrator = True)

DEBUG:openmm_propagators:initializing OMMLIAIS...
DEBUG:openmm_propagators:OMMLIAIS: metropolization is False
DEBUG:openmm_propagators:OMMLIAIS: successfully parsed splitting string
DEBUG:openmm_propagators:OMMLIAIS: adding global variables...
DEBUG:openmm_propagators:OMMLIAIS: adding integrator steps...
DEBUG:openmm_propagators:OMMLIAIS: adding substep functions...
DEBUG:openmm_reporters:creating trajectory storage object...
DEBUG:openmm_propagators:successfully executed ABCMeta init.
DEBUG:openmm_propagators:successfully equipped integrator: OMMLIAIS
DEBUG:openmm_propagators:integrator printable: None
  0%|          | 0/7 [00:00<?, ?it/s]DEBUG:openmm_propagators:	context_parameters before integration: {'lambda_angles': 0.0, 'lambda_bonds': 0.0, 'lambda_electrostatics_core': 0.0, 'lambda_electrostatics_delete': 0.0, 'lambda_electrostatics_insert': 0.0, 'lambda_sterics_core': 0.0, 'lambda_sterics_delete': 0.0, 'lambda_sterics_insert': 0.0, 'lambda_torsions': 0.0, 'softcore_alpha': 0.85

step      0 : allow forces to update the context state
step      1 : if(has_kT_changed = 1):
step      2 :    sigma <- sqrt(kT/m)
step      3 :    has_kT_changed <- 0
step      4 : end
step      5 : old_pe <- energy
step      6 : iteration <- iteration + 1.0
step      7 : fractional_iteration <- iteration / niterations
step      8 : lambda_sterics_core <- fractional_iteration
step      9 : lambda_electrostatics_core <- fractional_iteration
step     10 : lambda_sterics_insert <- select(step(fractional_iteration - 0.5), 1.0, 2.0 * fractional_iteration)
step     11 : lambda_sterics_delete <- select(step(fractional_iteration - 0.5), 2.0 * (fractional_iteration - 0.5), 0.0)
step     12 : lambda_electrostatics_insert <- select(step(fractional_iteration - 0.5), 2.0 * (fractional_iteration - 0.5), 0.0)
step     13 : lambda_electrostatics_delete <- select(step(fractional_iteration - 0.5), 1.0, 2.0 * fractional_iteration)
step     14 : lambda_bonds <- fractional_iteration
step     15 : lambda_an

Exception ignored in: <bound method PDBTrajectoryFile.__del__ of <mdtraj.formats.pdb.pdbfile.PDBTrajectoryFile object at 0x7f32d7972c88>>
Traceback (most recent call last):
  File "/home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/mdtraj/formats/pdb/pdbfile.py", line 666, in __del__
    self.close()
  File "/home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/mdtraj/formats/pdb/pdbfile.py", line 501, in close
    self._write_footer()
  File "/home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/mdtraj/formats/pdb/pdbfile.py", line 429, in _write_footer
    index1 = atomIndex[atom1]
KeyError: (MOL1-C1,)
DEBUG:openmm_propagators:	context_parameters after integration: {'lambda_angles': 1.0, 'lambda_bonds': 1.0, 'lambda_electrostatics_core': 1.0, 'lambda_electrostatics_delete': 1.0, 'lambda_electrostatics_insert': 1.0, 'lambda_sterics_core': 1.0, 'lambda_sterics_delete': 1.0, 'lambda_sterics_insert': 1.0, 'lambda_torsions': 1.0, 'softcore_alpha': 0.85}
 1

so it looks like there is a problem in the way that the initial energy of the non_initialized state is running...



In [7]:
ais_propagator._record_state_work_interval

1

In [8]:
ais_propagator._state_works

{0: [0.0,
  0.022771012095603636,
  0.04542108425547449,
  0.06793830396306942,
  0.09032358148025758,
  0.11258413308295875,
  0.13473952021375532,
  0.15681308573620328,
  0.17883320151842422,
  0.20082901333440645,
  0.22282559493870024,
  0.24484883555700795,
  0.2669275278421955,
  0.2890885082828328,
  0.31135181143606994,
  0.3337308087066543,
  0.3562340992795383,
  0.37886471797207993,
  0.4016166338751068,
  0.42447755118507763,
  0.4474256392228407,
  0.4704297926730548,
  0.493457033925662,
  0.5164663535972965,
  0.53941750364004,
  0.5622710745375423,
  0.5849944481536626,
  0.6075660055884495,
  0.6299733586680702,
  0.6522134857289378,
  0.6742885763366379,
  0.6962110343713758,
  0.7179978493338931,
  0.739668651928774,
  0.7612536722918333,
  0.7827865390192618,
  0.8043008893576779,
  0.8258329385363147,
  0.8474176949623468,
  0.8690851243364724,
  0.8908591768561547,
  0.9127556773233636,
  0.9347823558624201,
  0.9569419631501718,
  0.9792282060535212,
  1.0016262

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
for q in np.array(master_works):
    print(len(q))
    plt.plot(np.append(np.array([0.]), q))

In [ ]:
from openmmtools.states import *
from openmmtools.integrators import LangevinIntegrator
import openmmtools.cache
from perses.annihilation.lambda_protocol import RelativeAlchemicalState
thermostate = ThermodynamicState(hybrid_system, temperature=300*unit.kelvin, pressure=None)
alchemical_state = RelativeAlchemicalState.from_system(hybrid_system)
comp = CompoundThermodynamicState(thermostate, [alchemical_state])

integrator = LangevinIntegrator(temperature=thermostate.temperature)
context = comp.create_context(integrator)

positions1, positions2 = traj.xyz[33] * unit.nanometers, traj.xyz[45] * unit.nanometers
sampler_state1 = SamplerState(positions1, box_vectors = traj.unitcell_vectors[33]*unit.nanometers)
sampler_state2 = SamplerState(positions2, box_vectors = traj.unitcell_vectors[45]*unit.nanometers)



In [ ]:
n_steps = 10
x = 'fractional_iteration'
alchemical_functions = {
                         'lambda_sterics_core': x,
                         'lambda_electrostatics_core': x,
                         'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                         'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                         'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                         'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                         'lambda_bonds': x,
                         'lambda_angles': x,
                         'lambda_torsions': x}


AIS_integrator = OpenMMIAAISLangevinIntegrator(openmm_pdf_update_functions = alchemical_functions,
                                                n_steps = n_steps,
                                                temperature=pdf_state.temperature,
                                                collision_rate=1.0 / unit.picoseconds,
                                                timestep=4.0 * unit.femtoseconds,
                                                splitting="P I H N S V R O R V",
                                                constraint_tolerance=1e-6)



try:
    os.system(f"rm -r test_fluoro_neq")
except:
    pass

ais_propagator = OpenMMBaseIntegrationPropagator(openmm_pdf_state = pdf_state, integrator = AIS_integrator)
master_works = []
frames = [45, 33]
for num_trajectories in range(2):
    ais_propagator.integrator.reset()
    num_global_vars = ais_propagator.integrator.getNumGlobalVariables()
    global_vars = {ais_propagator.integrator.getGlobalVariableName(q): ais_propagator.integrator.getGlobalVariable(q) for q in range(num_global_vars)}
    print(global_vars)
    random_frame = frames[num_trajectories]
    positions = traj.xyz[random_frame] * unit.nanometers
    particle_state = OpenMMParticleState(positions=positions, box_vectors=traj.unitcell_vectors[random_frame]*unit.nanometers)
    particle_state.apply_to_context(ais_propagator.context)
    #pdf_state.apply_to_context(ais_propagator.context)
    context_params = ais_propagator.context.getParameters()
    context_params_nonswg = {q: context_params[q] for q in context_params}
    print(f"context_params: ",context_params_nonswg)
    #[ais_propagator.context.setGlobalVariableByName(key, 0.0) for key in global_vars.keys()]
    import tqdm
    works = []
    for _ in tqdm.trange(n_steps):
        _, return_dict = ais_propagator.apply(particle_state, n_steps=1, return_state_work=True, return_proposal_work=True,reset_integrator=False)
        num_global_vars = ais_propagator.integrator.getNumGlobalVariables()
        global_vars = {ais_propagator.integrator.getGlobalVariableName(q): ais_propagator.integrator.getGlobalVariable(q) for q in range(num_global_vars)}
        print(f"\titeration: {_}:")
        for key, item in global_vars.items():
            print(f"\t\t{key}: {item}")
        #reporter.record([particle])
        context_state = ais_propagator.context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                 enforcePeriodicBox=ais_propagator.pdf_state.is_periodic)
        pe  = context_state.getPotentialEnergy().value_in_unit(_OPENMM_ENERGY_UNIT)
        print(f"ENERGY_during: {pe}")
        works.append(return_dict['state_work'])
#     try:
#         reporter.record([particle], save_to_disk=True)
#     except Exception as e:
#         print(e)
    master_works.append(works)